In [1]:
import cv2
from skimage.feature import hog
import numpy as np
import os

# Define the image size and number of HOG cells
img_size = (200, 200)
cells_per_block = (2, 2)

# Define the HOG feature extractor
hog_desc = hog

# Define the dataset directories
train_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/edge/train/'
test_dir = '/Users/yashp/OneDrive/Desktop/RESEARCH/edge/test/'

# Define the anomaly class labels
anomaly_classes = ['good', 'anomaly']

# Load the training data
train_data = []
train_labels = []
for class_idx, class_name in enumerate(anomaly_classes):
    class_dir = os.path.join(train_dir, class_name)
    for filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)
        edges = cv2.Canny(img, 100, 200)
        hog_feats = hog_desc(edges, orientations=8, pixels_per_cell=(32, 32),
                             cells_per_block=cells_per_block, block_norm='L2-Hys')
        train_data.append(hog_feats)
        train_labels.append(class_idx)

# Load the testing data
test_data = []
test_labels = []
for class_idx, class_name in enumerate(anomaly_classes):
    class_dir = os.path.join(test_dir, class_name)
    for filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)
        edges = cv2.Canny(img, 100, 200)
        hog_feats = hog_desc(edges, orientations=8, pixels_per_cell=(32, 32),
                             cells_per_block=cells_per_block, block_norm='L2-Hys')
        test_data.append(hog_feats)
        test_labels.append(class_idx)

# Convert the data and labels to numpy arrays
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

# Train a Support Vector Machine (SVM) classifier
from sklearn.svm import SVC

svm = SVC(kernel='linear', C=1.0)
svm.fit(train_data, train_labels)

# Evaluate the performance of the model
from sklearn.metrics import accuracy_score, confusion_matrix

train_preds = svm.predict(train_data)
train_acc = accuracy_score(train_labels, train_preds)
train_cm = confusion_matrix(train_labels, train_preds)

test_preds = svm.predict(test_data)
test_acc = accuracy_score(test_labels, test_preds)
test_cm = confusion_matrix(test_labels, test_preds)

print(f"Train accuracy: {train_acc}")
print(f"Train confusion matrix:\n{train_cm}")
print(f"Test accuracy: {test_acc}")
print(f"Test confusion matrix:\n{test_cm}")


Train accuracy: 0.9655172413793104
Train confusion matrix:
[[150   0]
 [  7  46]]
Test accuracy: 0.80625
Test confusion matrix:
[[61  4]
 [27 68]]


In [4]:
import GPUtil
GPUtil.getAvailable()

[0]

In [3]:
pip install GPUtil

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7400 sha256=f0c89b8b5aceaafa8d4205631e3dd126026bff54aa78e789ccb5b2f5c3ccb276
  Stored in directory: c:\users\yashp\appdata\local\pip\cache\wheels\79\a7\0c\d53673ba405bcb572989c614f77c87833b03c3ca07d1f3e48e
Successfully built GPUtil
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
use_cuda = torch.cuda.is_available()

In [6]:
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

In [7]:
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ",device)

Device:  cpu
